 # Notebook description

Create a final report using data from the previously conducted data analysis. 

The report consists of three parts: `TOP report`, `Comparison` and `Day by day reliability`.

For this purpose, the dash library was used.


Notepad configuration

In [55]:
import psycopg2
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash_table import DataTable
import dash
from dash.dependencies import Input, Output
import os
from dotenv import  load_dotenv

In [ ]:
load_dotenv("D:\Python\devcontainer\.env") #loading postgres password from .env file

Create url and engine

In [57]:
con = psycopg2.connect(host='localhost',
                       user='postgres', 
                       password=os.getenv("POSTGRES_PASSWORD"))
cur = con.cursor()

 ## Creating a table for `TOP 10 routes`

Uploading data into the `top_routes_df` data frame from the `reporting.top_reliability_roads` view.

In [58]:
query = "SELECT * FROM reporting.top_reliability_roads"

cur.execute(query)

top_routes_df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

Update the numeric format for the `reliability` column.

In [59]:
top_routes_df['reliability'] = top_routes_df['reliability'].astype(float)

In [60]:
top_routes_df['reliability'] = (top_routes_df['reliability'] * 100).round(2)

Rename columns

In [61]:
top_routes_df = top_routes_df.drop(columns=['origin_airport_id', 'dest_airport_id', 'cnt', 'nb'])

In [62]:
top_routes_df = top_routes_df.sort_values(by='reliability', ascending=False).reset_index(drop=True)

In [63]:
top_routes_df['Rank'] = top_routes_df['reliability'].rank(ascending=False)

In [64]:
new_names = {
'origin_airport_name': 'Origin',
'dest_airport_name': 'Destination',
'year': 'Year',
'reliability': 'Reliability'
}

top_routes_df = top_routes_df.rename(columns=new_names)

In [65]:
top_routes_df = top_routes_df.head(10)

In [66]:
top_routes_df.head()

,Origin,Destination,Year,Reliability,Rank
0,Chicago O'Hare International,LaGuardia,2019,40.25,1.0
1,Los Angeles International,John F. Kennedy International,2019,38.37,2.0
2,Los Angeles International,McCarran International,2019,37.35,3.0
3,McCarran International,Los Angeles International,2019,36.34,4.0
4,Los Angeles International,San Francisco International,2019,35.32,5.0


## Year-on-year comparison 2019 vs. 2020

Uploading data into the `yoy_comparision_df` data frame from the `reporting.year_to_year_comparision` view.

In [67]:
query = "SELECT * FROM reporting.year_to_year_comparision"

cur.execute(query)

yoy_comparision_df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

Convert a data frame into a form that allows you to create a bar chart.

The result written to the variable `yoy_comparison_to_plot_df`.

In [68]:
yoy_comparision_df['reliability'] = yoy_comparision_df['reliability'].astype(float)
yoy_comparision_df['reliability'] = (yoy_comparision_df['reliability'] * 100).round(2)

In [69]:
yoy_comparision_table = yoy_comparision_df.pivot_table(index='month', columns='year', values='reliability', fill_value=None)

Create a bar chart

In [70]:
yoy_comparision_fig = px.bar(yoy_comparision_table, 
             x=yoy_comparision_table.index, 
             y=[2019, 2020], 
             barmode='group',
             title='Comparison of reliability for 2019 and 2020',
             )

yoy_comparision_fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Reliability [%]",
    legend_title="Year",)

yoy_comparision_fig.show()

## Weekday to weekday comparison 2019 vs. 2020.

Uploading data into the `day_to_day_comparision_df` data frame from the `reporting.day_to_day_comparision` view.

In [71]:
query = "SELECT * FROM reporting.day_to_day_comparision"

cur.execute(query)

day_to_day_comparision_df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

In [72]:
day_to_day_comparision_df.head()

,year,day_of_week,flights_amount
0,2019,1,167761
1,2019,2,165478
2,2019,3,163889
3,2019,4,166154
4,2019,5,167397


Convert a data frame into a form that allows you to create a bar chart.

The result written to the variable `day_to_day_comparision_to_plot_df`.

In [73]:
day_to_day_comparision_to_plot_df = day_to_day_comparision_df.pivot_table(index='day_of_week', columns='year', values='flights_amount', fill_value=None)

Create a bar chart

In [74]:
day_to_day_comparision_to_plot_df.head()

year,2019,2020
day_of_week,,
1,167761.0,40983.0
2,165478.0,39807.0
3,163889.0,39581.0
4,166154.0,41086.0
5,167397.0,41058.0


In [75]:
day_to_day_comparision_fig = px.bar(
            day_to_day_comparision_to_plot_df, 
             x=day_to_day_comparision_to_plot_df.index, 
             y=[2019, 2020], 
             barmode='group',
             title='Flights amount at day for 2019 and 2020',
             )

day_to_day_comparision_fig.update_layout(
    xaxis_title="Day",
    yaxis_title="Flights amount",
    legend_title="Year",)

day_to_day_comparision_fig.show()

## Creating a time series

Uploading data into the `day_by_day_reliability_df` data frame from the  `reporting.day_by_day_reliability` view.

In [76]:
query = "SELECT * FROM reporting.day_by_day_reliability"

cur.execute(query)

day_by_day_reliability_df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

In [77]:
day_by_day_reliability_df['reliability'] = day_by_day_reliability_df['reliability'].astype(float)
day_by_day_reliability_df['reliability'] = (day_by_day_reliability_df['reliability'] * 100).round(2)

In [78]:
day_by_day_reliability_df['date'] = pd.to_datetime(day_by_day_reliability_df['date'])

day_by_day_reliability_df['year'] = day_by_day_reliability_df['date'].dt.year

Creating a line graph.

In [79]:
day_by_day_reliability_fig = px.line(
    day_by_day_reliability_df,
    x= 'date',
    y= 'reliability',
    color= 'year',
    title= 'Reliability for 2019 and 2020',
    labels={'date': 'Date', 'reliability': 'Reliability [%]', 'year': 'Year'}
)

day_by_day_reliability_fig.show()

## Creation of layouts for individual pages

### Creating a layout for the view `TOP 10 routes`.

Create the `Dash` `DataTable` component and save to the `top_routes_table` variable.

In [80]:
top_routes_table = DataTable(
    id = 'top_routes_table',
    columns= [{'name': i, 'id': i} for i in top_routes_df.columns],
    data= top_routes_df.to_dict('records')
)

Create a header for tebels and save to the `top_routes_page_title` variable.

In [81]:
top_routes_page_title = html.H3(
    'TOP 10 reliability routes in 2019 and 2020',
    style={'font-family': 'verdana', 'color': '#444444'},
    id='Top Routes'
)

Creating a layout for the view `TOP 10 routes`and save to the `top_routes_layout` variable.

In [82]:
top_routes_layout = html.Div([
    top_routes_page_title,
    top_routes_table,
])

### Creating a layout for the view `Comparision`.

Create the `Dash` `yoy_comparision_fig` component and save to the `yoy_comparision_component` variable.

In [83]:
yoy_comparision_component = dcc.Graph(figure= yoy_comparision_fig)

Create the `Dash` `day_to_day_comparision_fig` component and save to the `day_to_day_comparision_component` variable.

In [84]:
day_to_day_comparision_component = dcc.Graph(figure= day_to_day_comparision_fig)

Create the `layout` for the view `comparision` and save to the `comparision_layout` variable.

In [85]:
comparision_layout = html.Div([
    html.H3('Comparision', id='Comparision'),
    yoy_comparision_component 
])

Create the `layout` for the view`day by day` and save to the `day_by_day_layout` variable.

In [86]:
day_by_day_layout = html.Div([
    html.H3('Day by Day', id='Day by Day'),
    day_to_day_comparision_component
])

 # Configuring the application


In [90]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Nav([
        dcc.Link('TOP Report', href='/'),
        html.Span(' | '),
        dcc.Link('Comparison', href='/comparision_reporting'),
        html.Span(' | '),
        dcc.Link('Day by Day Reporting', href='/day_by_day_reporting'),
    ]),
    html.Div(id='page-content')
])

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def page(pathname):
    if pathname == '/comparision_reporting':
        return comparision_layout
    elif pathname == '/day_by_day_reporting':
        return day_by_day_layout
    else:
        return top_routes_layout

if __name__ == '__main__':
    app.run_server(debug=True)

 # Podsumowanie
 W tym notatniku stworzyliśmy zgodnie z wymaganiami raport do dyspozycji osób decyzyjnych. Kod źródłowy może w tym momencie zostać przekazany dalej, do działu IT, który następnie wdroży rozwiązanie na serwer dostępny dla każdej zainteresowanej osoby. W praktyce oznacza to koniec naszej pracy nad tym zadaniem. Choć warto dodać, że często po udostępnieniu raportu pojawiają się dodatkowe wymagania oraz komentarze ze względu na informacje, które są tam zawarte.

 W kolejnym notatniku podsumujemy sobie cały warsztat.